# Segmenting and Clustering Neighborhoods in Toronto

Using this Notebook to build the code to scrape the following Wikipedia page: 
    https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

First we will import the libraries that will be required for the task:

In [3]:
!conda install -c conda-forge wikipedia --yes

import pandas as pd 
import wikipedia as wp
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    wikipedia:       1.4.0-py_2        conda-forge

The following packages will be UPDATED:

    certifi:         2019.9.11-py36_0              --> 2019.9.11-py36_0     conda-forge

The following packages will be DOWNGRADED:


In order to obtain the data from the page we use wikipedia library to store the content in bytes.

In [6]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
html

b'<div class="mw-parser-output"><p>This is a list of <a href="/wiki/Postal_codes_in_Canada" title="Postal codes in Canada">postal codes in Canada</a> where the first letter is M. Postal codes beginning with M are located within the city of <a href="/wiki/Toronto" title="Toronto">Toronto</a> in the province of <a href="/wiki/Ontario" title="Ontario">Ontario</a>. Only the first three characters are listed, corresponding to the Forward Sortation Area.\n</p><p><a href="/wiki/Canada_Post" title="Canada Post">Canada Post</a> provides a free postal code look-up tool on its website,<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">&#91;1&#93;</a></sup> via its <a href="/wiki/Mobile_app" title="Mobile app">applications</a> for such <a href="/wiki/Smartphones" class="mw-redirect" title="Smartphones">smartphones</a> as the <a href="/wiki/IPhone" title="IPhone">iPhone</a> and <a href="/wiki/BlackBerry" title="BlackBerry">BlackBerry</a>,<sup id="cite_ref-2" class="reference"><a href="#c

Now as we see in that html the required table is at 0 index so we get the table by:

In [18]:
df = pd.read_html(html, header = 0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


To remove all the cells with not-assigned values we use:

In [17]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


As More than one neighborhood can exist in one postal code area. So the two neighnourhoods will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [16]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


So if the neighbourhood has "Not Assigned" we will change it to Borough.

In [15]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


So finally we have cleaned the data for this week. We can now move towards applying K-Mean Clustering.

In [19]:
df.shape

(288, 3)